In [1]:
import sys
import os

# Get the parent directory of the current folder
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t
from scipy.special import logsumexp, gamma, gammaln
from numpy.linalg import inv, det, slogdet, LinAlgError
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import tril_indices, triu_indices
from typing import Literal, Optional, Dict, Union

# from simulation import * 

from Functions.simulation_1 import * 
# from Functions.gas_filter_2 import *
# from Functions.kf_filter_2 import *
from Functions.gas_filter_5 import *
from Functions.kf_filter_5 import *

ModuleNotFoundError: No module named 'Functions'

## $\Phi$ Full, $Q, R$ Full

In [ ]:
T = 500
N = 3
K = 3

c_true_3x3 = np.array([0.2, -0.1, 0.4])

Phi_true_3x3_phi_full_yes_noise_correl = np.array([[0.8, 0.2, 0.1],
                                                    [0.1, 0.5, 0.1],
                                                    [0.1, 0.2, 0.6]])

beta_true_3x3 = np.array([[1.0, -0.5, 0.2],
                          [-1.2, 0.8, 0.4],
                          [0.5, 1.5, -0.3]])

Q_true_3x3_yes_correl = np.array([[0.5, 0.1, 0.3],
                                 [0.1, 0.5, 0.1],
                                 [0.3, 0.1, 0.5]])

R_true_3x3_yes_correl = np.array([[0.4, 0.1, 0.2],
                                 [0.1, 0.5, 0.3],
                                 [0.2, 0.3, 0.5]])


y_sim_3x3_phi_full_yes_noise_correl, mu_sim_3x3_phi_full_yes_noise_correl, X_sim_3x3_phi_full_yes_noise_correl = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true_3x3,
    Phi=Phi_true_3x3_phi_full_yes_noise_correl,
    beta=beta_true_3x3, 
    Q=Q_true_3x3_yes_correl,
    R=R_true_3x3_yes_correl,
    seed=8888
)

In [4]:
eigvals = np.linalg.eigvals(Phi_true_3x3_phi_full_yes_noise_correl)
spectral_radius = np.max(np.abs(eigvals))
print("Eigenvalues:", eigvals)
print("Spectral radius:", spectral_radius)

Eigenvalues: [0.92491405 0.58536345 0.3897225 ]
Spectral radius: 0.9249140538129542


In [ ]:
initial_params_3x3_phi_full_yes_noise_correl = np.concatenate([
    c_true_3x3, 
    Phi_true_3x3_phi_full_yes_noise_correl.flatten(), 
    beta_true_3x3.flatten(), 
    np.ones(N * (N + 1) // 2),  # L_Q_elements
    np.ones(N * (N + 1) // 2)   # L_R_elements
])

kf_results_3x3_phi_full_yes_correl  =  multivariate_KF_with_estimation(
    y=y_sim_3x3_phi_full_yes_noise_correl,
    X=X_sim_3x3_phi_full_yes_noise_correl,
    verbose=True,
    ftol=1e-8,
    gtol=1e-6,
)

mu_kf_3x3_phi_full_yes_correl = kf_results_3x3_phi_full_yes_correl['mu_filtered']
P_kf_3x3_phi_full_yes_correl = kf_results_3x3_phi_full_yes_correl['P_filtered']
kalman_gain_kf_3x3_phi_full_yes_correl = kf_results_3x3_phi_full_yes_correl['kalman_gain']

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [0.00864088 0.49023981 0.34229745]
Phi (persistence matrix): 
[[ 0.89417749  0.12109202  0.05550341]
 [ 0.19589246  0.53900264 -0.03975424]
 [ 0.15997166  0.22488049  0.55068365]]
beta: 
[[ 1.85269823 -0.86515753  0.82268523]
 [-1.22167628  0.80876146  0.42471769]
 [ 0.50066157  1.5258856  -0.25391934]]
Q (state noise covariance): 
[[0.38542034 0.1526911  0.29485396]
 [0.1526911  0.34965434 0.13182503]
 [0.29485396 0.13182503 0.48205983]]
R (observation noise covariance): 
[[0.49182195 0.0007013  0.22332987]
 [0.0007013  0.68448753 0.30960301]
 [0.22332987 0.30960301 0.55219684]]
Log-likelihood: -2079.8614824305732
AIC: 4225.7229648611465
BIC: 4401.059237635126


-2122.8821446541792

-2274.143212422688

autoregressive matrix, gives the impulse response
VARMA literature
specification of the phi, kappa matrix

In [6]:
gas_results_3x3_phi_full_yes_correl_1 = estimate_and_filter_gas(
                                        y=y_sim_3x3_phi_full_yes_noise_correl, 
                                        X=X_sim_3x3_phi_full_yes_noise_correl, 
                                        phi_type= "full", 
                                        kappa_type= "full", 
                                        fix_nu= None,
                                        ftol = 1e-8,  # Tolerance for convergence
                                        gtol = 1e-5,   # Gradient tolerance for convergence)
)


mu_gas_3x3_phi_full_yes_correl_1 = gas_results_3x3_phi_full_yes_correl_1["mu_filtered"]
u_gas_3x3_phi_full_yes_correl_1 = gas_results_3x3_phi_full_yes_correl_1["u"]
resid_gas_phi_full_yes_correl_1 = gas_results_3x3_phi_full_yes_correl_1["resid"]
Kappa_est_3x3_phi_full_yes_correl_1 = gas_results_3x3_phi_full_yes_correl_1["Kappa"]

Estimated omega:
 [0.6636918  0.1355977  0.33629389]
Estimated Phi:
 [[0.80918078 0.05602541 0.18177033]
 [0.01996055 0.59373259 0.17974211]
 [0.11572004 0.02605674 0.68873474]]
Estimated beta:
 [[ 3.03676197  0.22243729  1.92203399]
 [-1.21217225  0.80986906  0.45951294]
 [ 0.49863399  1.52728007 -0.26149758]]
Estimated Omega (from lambda):
 [[1.02121724 0.         0.        ]
 [0.         1.05731633 0.        ]
 [0.         0.         1.10209251]]
Estimated Kappa:
 [[ 0.60622931  0.15394278  0.07596483]
 [ 0.25258616  0.27736129 -0.11912907]
 [ 0.27544007  0.12318373  0.29152165]]
Estimated nu:
 31.08906986795939
Log-likelihood: -2219.7200431385368
AIC: 4507.4400862770735
BIC: 4688.089579438144


In [7]:
test1 = mu_sim_3x3_phi_full_yes_noise_correl[1:]
test2 = mu_kf_3x3_phi_full_yes_correl[1:]
test3 = mu_gas_3x3_phi_full_yes_correl_1[1:]

print(np.mean((test1 - test2) ** 2))
print(np.mean((test1 - test3) ** 2))

0.760474923651199
3.0494134780169118


In [ ]:
gas_specs = [
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_diag_diag_nufree"},
    # {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 5, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_diag_diag_nu20"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_diag_diag_nu50"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_diag_diag_nu100"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_diag_diag_nu500"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": None, "label": "GAS_diag_full_nufree"},
    # {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_full_nu5"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 10, "label": "GAS_diag_full_nu10"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 20, "label": "GAS_diag_full_nu20"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 50, "label": "GAS_diag_full_nu50"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 100, "label": "GAS_diag_full_nu100"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 500, "label": "GAS_diag_full_nu500"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_full_diag_nufree"},
    # {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 5, "label": "GAS_full_diag_nu5"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_full_diag_nu10"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_full_diag_nu20"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_full_diag_nu50"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_full_diag_nu100"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_full_diag_nu500"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": None, "label": "GAS_full_full_nufree"},
    # {"phi_type": "full", "kappa_type": "full", "fix_nu": 5, "label": "GAS_full_full_nu5"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 10, "label": "GAS_full_full_nu10"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 20, "label": "GAS_full_full_nu20"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 50, "label": "GAS_full_full_nu50"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 100, "label": "GAS_full_full_nu100"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 500, "label": "GAS_full_full_nu500"},
]

In [ ]:
from collections import defaultdict

T = 500


spec_results = defaultdict(list)
target_success = 30
sim_counter = 0

while any(len(v) < target_success for v in spec_results.values()) or len(spec_results) == 0:
    sim_seed = 8888 + sim_counter
    sim_counter += 1
    # Simulate data
    y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
            T=1000,
            N=N,
            c=c_true_3x3,
            Phi=Phi_true_3x3_phi_full_yes_noise_correl,
            beta=beta_true_3x3, 
            Q=Q_true_3x3_yes_correl,
            R=R_true_3x3_yes_correl,
            seed=sim_seed,
        )
    # KF
    try:
        if len(spec_results["KF"]) < target_success:
            kf_results = multivariate_KF_with_estimation(
            y=y_sim, 
            X=X_sim, 
            initial_params=None, 
            verbose=False,
            ftol=1e-8,  # Tolerance for convergence
            gtol=1e-5,  # Gradient tolerance for convergence,
        )
            # Calculate MSE for mu
            mu_kf = kf_results['mu_filtered']
            mse_kf = np.mean((mu_sim[1:] - mu_kf[1:]) ** 2)
            mae_kf = np.mean(np.abs(mu_sim[1:] - mu_kf[1:]))

            spec_results["KF"].append({
                "seed": sim_seed, "loglik": kf_results["loglik"],
                "aic": kf_results["aic"], "bic": kf_results["bic"],
                "mse": mse_kf, "mae": mae_kf
                })

    except Exception as e:
        print(f"KF failed: {e}")
    # GAS
    for spec in gas_specs:
        try:
            label = spec["label"]
            if len(spec_results[label]) < target_success:
                res_gas = estimate_and_filter_gas(
                    y=y_sim, 
                    X=X_sim,
                    phi_type=spec["phi_type"], 
                    kappa_type=spec["kappa_type"],
                    fix_nu=spec["fix_nu"], 
                    verbose=False,
                    ftol=1e-8,  # Tolerance for convergence
                    gtol=1e-5   # Gradient tolerance for convergence
                )

                # Calculate MSE for mu
                mu_gas = res_gas["mu_filtered"]
                mse_gas = np.mean((mu_sim[1:] - mu_gas[1:]) ** 2)
                mae_gas = np.mean(np.abs(mu_sim[1:] - mu_gas[1:]))

                spec_results[label].append({
                    "seed": sim_seed, "loglik": res_gas["loglik"],
                    "aic": res_gas["aic"], "bic": res_gas["bic"],
                    "mse": mse_gas, "mae": mae_gas
                })
        except Exception as e:
            print(f"GAS {label} failed: {e}")

    # Break if all filled
    if all(len(v) >= target_success for v in spec_results.values()):
        break


In [ ]:
import pandas as pd

# Flatten to long-form list of dicts
rows = []
for spec_label, run_list in spec_results.items():
    for run in run_list:
        row = {'spec': spec_label}
        row.update(run)  # adds 'seed', 'loglik', etc.
        rows.append(row)

df_results = pd.DataFrame(rows)


agg = df_results.groupby('spec').agg({
    'loglik': ['mean', 'std'],
    'aic': ['mean', 'std'],
    'bic': ['mean', 'std'],
    'mse': ['mean', 'std'], 
    'mae': ['mean', 'std'],  
}).reset_index()

agg.columns = ['spec', 'loglik_mean', 'loglik_std', 'aic_mean', 'aic_std', 'bic_mean', 'bic_std', 'mse_mean', 'mse_std', 'mae_mean', 'mae_std']  # Added mse and mae


agg

,spec,loglik_mean,loglik_std,aic_mean,aic_std,bic_mean,bic_std,mse_mean,mse_std,mae_mean,mae_std
0,GAS_diag_diag_nu10,-4503.872911,43.918616,9049.745823,87.837231,9175.879542,87.837231,3.315474,0.518791,1.612231,0.135370
1,GAS_diag_diag_nu100,-4479.526337,42.858266,9001.052675,85.716532,9127.186394,85.716532,1.878742,0.276375,1.213456,0.100141
2,GAS_diag_diag_nu20,-4483.861115,43.265052,9009.722230,86.530105,9135.855948,86.530105,2.804379,0.464308,1.487370,0.132428
3,GAS_diag_diag_nu50,-4478.941700,42.945599,8999.883400,85.891199,9126.017119,85.891199,2.146285,0.318201,1.300581,0.107037
4,GAS_diag_diag_nu500,-4481.248113,42.786697,9004.496225,85.573393,9130.629944,85.573393,1.648665,0.222256,1.132530,0.087334
5,GAS_diag_diag_nufree,-4478.375726,42.809329,9000.751451,85.618658,9132.891538,85.618658,17.715706,15.774659,3.342253,1.322803
6,GAS_diag_full_nu10,-4450.084481,42.196427,8954.168961,84.392854,9116.340885,84.392854,3.347945,0.541651,1.616946,0.140026
7,GAS_diag_full_nu100,-4438.545638,41.266341,8931.091276,82.532681,9093.263200,82.532681,1.863135,0.259276,1.201860,0.094767
8,GAS_diag_full_nu20,-4436.326500,41.597475,8926.652999,83.194950,9088.824924,83.194950,2.795079,0.457577,1.480856,0.131096
9,GAS_diag_full_nu50,-4436.116665,41.323355,8926.233329,82.646710,9088.405253,82.646710,2.112647,0.318323,1.283973,0.106822


In [11]:
print(round(agg, 2).to_latex())

\begin{tabular}{llrrrrrrrrrr}
\toprule
 & spec & loglik_mean & loglik_std & aic_mean & aic_std & bic_mean & bic_std & mse_mean & mse_std & mae_mean & mae_std \\
\midrule
0 & GAS_diag_diag_nu10 & -4503.870000 & 43.920000 & 9049.750000 & 87.840000 & 9175.880000 & 87.840000 & 3.320000 & 0.520000 & 1.610000 & 0.140000 \\
1 & GAS_diag_diag_nu100 & -4479.530000 & 42.860000 & 9001.050000 & 85.720000 & 9127.190000 & 85.720000 & 1.880000 & 0.280000 & 1.210000 & 0.100000 \\
2 & GAS_diag_diag_nu20 & -4483.860000 & 43.270000 & 9009.720000 & 86.530000 & 9135.860000 & 86.530000 & 2.800000 & 0.460000 & 1.490000 & 0.130000 \\
3 & GAS_diag_diag_nu50 & -4478.940000 & 42.950000 & 8999.880000 & 85.890000 & 9126.020000 & 85.890000 & 2.150000 & 0.320000 & 1.300000 & 0.110000 \\
4 & GAS_diag_diag_nu500 & -4481.250000 & 42.790000 & 9004.500000 & 85.570000 & 9130.630000 & 85.570000 & 1.650000 & 0.220000 & 1.130000 & 0.090000 \\
5 & GAS_diag_diag_nufree & -4478.380000 & 42.810000 & 9000.750000 & 85.620000 & 913